# Homework and bakeoff: Few-shot OpenQA with DSPy

In [1]:
__author__ = "Christopher Potts and Omar Khattab"
__version__ = "CS224u, Stanford, Spring 2024"

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/cgpotts/cs224u/blob/master/hw_openqa.ipynb)
[![Open in SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/cgpotts/cs224u/blob/master/hw_openqa.ipynb)

If Colab is opened with this badge, please **save a copy to drive** (from the File menu) before running the notebook.

## Overview

The goal of this homework is to explore retrieval-augmented in-context learning. This is an exciting area that brings together a number of recent task ideas and modeling innovations. We will use the [DSPy programming library](http://dspy.ai) to build systems in this new mode.

Our core task is __open-domain question answering (OpenQA)__. In this task, all that is given by the dataset is a question text, and the task is to answer that question. By contrast, in many modern QA tasks, the dataset provides a text and a gold passage, usually with a firm guarantee that the answer will be a substring of the passage.

OpenQA is substantially harder than standard QA. The usual strategy is to use a _retriever_ to find passages in a large collection of texts and train a _reader_ to find answers in those passages. This means we have no guarantee that the retrieved passage will contain the answer we need. If we don't retrieve a passage containing the answer, our reader has no hope of succeeding. Although this is challenging, it is much more realistic and widely applicable than standard QA. After all, with the right retriever, an OpenQA system could be deployed over the entire Web.

The task posed by this homework is harder even than OpenQA. We are calling this task __few-shot OpenQA__. The defining feature of this task is that the reader is simply a frozen, general purpose language model. It accepts string inputs (prompts) and produces text in response. It is not trained to answer questions per se, and nothing about its structure ensures that it will respond with a substring of the prompt corresponding to anything like an answer.

__Few-shot QA__ (but not OpenQA!) is explored in the famous GPT-3 paper ([Brown et al. 2020](https://arxiv.org/abs/2005.14165)). The authors are able to get traction on the problem using GPT-3, an incredible finding. Our task here – __few-shot OpenQA__ – pushes this even further by retrieving passages to use in the prompt rather than assuming that the gold passage can be used in the prompt. If we can make this work, then it should be a major step towards flexibly and easily deploying QA technologies in new domains.

In summary:

| Task             | Passage given | Task-specific reader training |Task-specific retriever training  | 
|-----------------:|:-------------:|:-----------------------------:|:--------------------------------:|
| QA               | yes           | yes                           | n/a                              |
| OpenQA           | no            | yes                           | maybe                            |
| Few-shot QA      | yes           | no                            | n/a                              |
| Few-shot OpenQA  | no            | no                            | maybe                            | 

Just to repeat: your mission is to explore the final line in this table. The core notebook and assignment don't address the issue of training the retriever in a task-specific way, but this is something you could pursue for a final project; [the ColBERT codebase](https://github.com/stanford-futuredata/ColBERT) makes easy.

It is a requirement of the bake-off that a general-purpose language model be used. In particular, trained QA systems cannot be used at all, and no fine-tuning is allowed either. See the original system question at the bottom of this message for guidance on which models are allowed.

Note: the models we are working with here are _big_. This poses a challenge that is increasingly common in NLP: you have to pay one way or another. You can pay to use the GPT-3 API, or you can pay to use a local model on a heavy-duty cluster computer, or you can pay with time by using a local model on a more modest computer.

## Set-up

We have sought to make this notebook self-contained and easy to use on a personal computer, on Google Colab, and in Sagemaker Studio. For personal computer use, we assume you have already done everything in [setup.ipynb](setup.ipynb]). For cloud usage, the next few code blocks should handle all set-up steps.

In [2]:
try:
    # This library is our indicator that the required installs
    # need to be done.
    import datasets
    root_path = '.'
except ModuleNotFoundError:
    !git clone https://github.com/cgpotts/cs224u/
    !pip install -r cs224u/requirements.txt
    root_path = 'dspy'

In [3]:
from datasets import load_dataset
import openai
import os
import dspy
from dotenv import load_dotenv

Save the API keys in a `.env` file in the local root directory as follows. Then, `load_dotenv()` will make them available to the notebook:

In [4]:
# keep the API keys in a `.env` file in the local root directory
load_dotenv()

True

In [5]:
os.environ["DSP_NOTEBOOK_CACHEDIR"] = os.path.join(root_path, 'cache')

openai_key = os.getenv('OPENAI_API_KEY')  # or replace with your API key (optional)

colbert_server = 'http://index.contextual.ai:8893/api/search'

Here we establish the Language module `lm` and Retriever module `rm` that we will be using. The defaults for `lm` are just for development. You may want to develop using an inexpensive model and then do your final evalautions wih an expensive one. DSPy has support for a wide range of model APIs and local models.

In [6]:
lm = dspy.OpenAI(model='gpt-3.5-turbo', api_key=openai_key)

rm = dspy.ColBERTv2(url=colbert_server)

dspy.settings.configure(lm=lm, rm=rm)

Here's a command you can run to see which OpenAI models are available; OpenAI has entered into an increasingly closed mode where many older models are not available, so there are likely to be some surprises lurking here:

In [7]:
[d['id'] for d in openai.Model.list()['data']]
# Adapt to current API
# client = openai.OpenAI(api_key=openai_key)
# [d.id for d in client.models.list().data]

['gpt-4-vision-preview',
 'dall-e-3',
 'gpt-3.5-turbo-0613',
 'text-embedding-3-large',
 'gpt-3.5-turbo-instruct-0914',
 'dall-e-2',
 'whisper-1',
 'tts-1-hd-1106',
 'tts-1-hd',
 'babbage-002',
 'text-embedding-ada-002',
 'gpt-3.5-turbo-0125',
 'gpt-3.5-turbo',
 'text-embedding-3-small',
 'gpt-3.5-turbo-0301',
 'gpt-3.5-turbo-instruct',
 'gpt-3.5-turbo-16k',
 'tts-1',
 'tts-1-1106',
 'gpt-4-0125-preview',
 'gpt-3.5-turbo-1106',
 'gpt-4',
 'gpt-4-turbo-preview',
 'gpt-4-0613',
 'gpt-3.5-turbo-16k-0613',
 'davinci-002',
 'gpt-4-1106-preview']

## SQuAD

Our core development dataset is [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/). We chose this dataset because it is well-known and widely used, and it is large enough to support lots of meaningful development work, without, though, being so large as to require lots of compute power.

In [8]:
squad = load_dataset("squad")

The following utility just reads a SQuAD split in as a list of `dspy.Example` instances:

In [9]:
def get_squad_split(squad, split="validation"):
    """
    Use `split='train'` for the train split.

    Returns
    -------
    list of dspy.Example with attributes question, answer

    """
    data = zip(*[squad[split][field] for field in squad[split].features])
    exs = [dspy.Example(question=q, answer=a['text'][0]).with_inputs("question")
           for eid, title, context, q, a in data]
    return exs

### SQuAD train

To build few-shot prompts, we will often sample SQuAD train examples, so we load that split here:

In [10]:
squad_train = get_squad_split(squad, split="train")

### SQuAD dev

In [11]:
squad_dev = get_squad_split(squad)

In [12]:
len(squad_dev)

10570

### SQuAD dev sample

Evaluations are expensive in this new era! Here's a small sample to use for dev assessments:

In [13]:
import random

random.seed(1)

dev_exs = random.sample(squad_dev, k=200)

## DSPy basics

### LM usage

Here's the most basic way to use the LM:

In [14]:
lm("Which award did Gary Zukav's first book receive?")

['Gary Zukav\'s first book, "The Dancing Wu Li Masters: An Overview of the New Physics," received the 1979 American Book Award for Science.']

Keyword arguments to the underlying LM are passed through:

In [15]:
lm("Which U.S. states border no U.S. states?", temperature=0.9, n=4)

['Hawaii and Alaska are the two U.S. states that do not share a border with any other U.S. state.',
 'Hawaii and Alaska do not border any other U.S. states.',
 'Hawaii and Alaska are the only two U.S. states that do not share a border with any other U.S. state.',
 'There are three U.S. states that do not share a border with any other U.S. states. They are Alaska, Hawaii, and Connecticut. Alaska and Hawaii are both located outside the contiguous United States, while Connecticut is located in New England and is surrounded by the states of Massachusetts, New York, and Rhode Island.']

With `lm.inspect_history`, we can see the most recent language model calls:

In [16]:
lm.inspect_history(n=1)





Which U.S. states border no U.S. states? Hawaii and Alaska are the two U.S. states that do not share a border with any other U.S. state. 	 (and 3 other completions)





### Signature-based prediction

In DSPy, __signatures__ are declarative statements about what we want the model to do. In the following `"question -> answer"` is the signature (the most basic QA signature one could write), and `dspy.Predict` is used to turn this into a complete QA system: 

In [17]:
basic_predictor = dspy.Predict("question -> answer")

Here we use `basic_predictor`:

In [18]:
basic_predictor(question="Which award did Gary Zukav's first book receive?")

Prediction(
    answer="Question: Which award did Gary Zukav's first book receive?\nAnswer: The Seat of the Soul received the 1989 Book of the Year award from the American Book Awards."
)

And here is the prompt that was given to the model:

In [19]:
lm.inspect_history(n=1)





Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Answer: ${answer}

---

Question: Which award did Gary Zukav's first book receive?
Answer: Question: Which award did Gary Zukav's first book receive?
Answer: The Seat of the Soul received the 1989 Book of the Year award from the American Book Awards.





In many cases, we will want more control over the prompt. Writing a small custom `dspy.Signature` class is the easiest way to accomplish this. In the following, we just just tweak the initial instruction and provide some formatting guidance for the answer:

In [20]:
class BasicQASignature(dspy.Signature):
    __doc__ = """Answer questions with short factoid answers."""

    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

In [21]:
sig_predictor = dspy.Predict(BasicQASignature)

In [22]:
sig_predictor(question="Which U.S. states border no U.S. states?")

Prediction(
    answer='Maine, Hawaii'
)

In [23]:
lm.inspect_history(n=1)





Answer questions with short factoid answers.

---

Follow the following format.

Question: ${question}
Answer: often between 1 and 5 words

---

Question: Which U.S. states border no U.S. states?
Answer: Maine, Hawaii





### Modules

One of the hallmarks of DSPy is that it adopts design patterns from PyTorch. The main example of this is DSPy's use of the `Module` as the basic unit for writing simple and complex programs. Here is a very basic module for QA that makes use of `BasicQASignature` as we defined it just above.

In [24]:
class BasicQA(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.Predict(BasicQASignature)

    def forward(self, question):
        return self.generate_answer(question=question)

As with PyTorch, the `forward` methos is called when we want to make predictions:

In [25]:
basic_qa_model = BasicQA()

In [26]:
basic_qa_model(question="Which award did Gary Zukav's first book receive?")

Prediction(
    answer='National Book Award'
)

The modular design of DSPy starts to become apparent now. If you want to change the above to use chain of thought instead of regular predictions, you need only change `dspy.Predict` to `dspy.ChainOfThought`, and similarly for `dspy.ReAct`, `dspy.ProgramOfThought`, or a module you wrote yourself.

### Teleprompting

The QA system we've defined so far is a zero-shot system. To change it into a few-shot system, we will rely on a DSPy __teleprompter__. This will allow us to flexibly move between the zero-shot and few-shot formulations. The following code achieves this.

In [27]:
from dspy.teleprompt import LabeledFewShot

./cache/compiler


Here we instantiate a `LabeledFewShot` teleprompter that will add three demonstrations. These will be sampled randomly from the set of train examples we provide:

In [28]:
fewshot_teleprompter = LabeledFewShot(k=3)

And then we call `compile` on `basic_qa_model` as we defined it above. This returns a new module that we use like any other in DSPy:

In [29]:
basic_fewshot_qa_model = fewshot_teleprompter.compile(basic_qa_model, trainset=squad_train)

In [30]:
basic_fewshot_qa_model(question="Which award did Gary Zukav's first book receive?")

Prediction(
    answer='American Book Award'
)

With `inspect_history`, we can see that prompts now contain demonstrations:

In [31]:
lm.inspect_history(n=1)





Answer questions with short factoid answers.

---

Follow the following format.

Question: ${question}
Answer: often between 1 and 5 words

---

Question: What group did Paul VI address in New York in 1965?
Answer: United Nations

---

Question: What did Sander's study show in terms of black law students rankings?
Answer: half of all black law students rank near the bottom of their class after the first year of law school

---

Question: What problems does linguistic anthropology bring linguistic methods to bear on?
Answer: anthropological

---

Question: Which award did Gary Zukav's first book receive?
Answer: American Book Award





### Evaluation

Our evaluation metric is a standard one for SQuAD and related tasks: exact match of the answer (EM).

In [32]:
from dspy.evaluate import answer_exact_match
from dspy.evaluate.evaluate import Evaluate

In [33]:
answer_exact_match(dspy.Example(answer="STAGE 2!"), dspy.Prediction(answer="stage 2"))

True

In DSPy, `Evaluate` objects provide a uniform interface for running evaluations. Here are two for us to use in development. The first will evaluate on all of `dev_exs` and should provide a meaningful picture of how a system is doing. It could be expensive to use it a lot, though. The second is for debugging and is probably too small to give a reliable estimate.

In [34]:
dev_evaluater = Evaluate(
    devset=dev_exs, # 200 examples
    num_threads=1,
    display_progress=True,
    display_table=5)

In [35]:
tiny_evaluater = Evaluate(
    devset=dev_exs[: 15],
    num_threads=1,
    display_progress=True,
    display_table=5)

Here is a tiny (debugging-oriented) evaluation of our few-shot QA sytem:

In [36]:
tiny_evaluater(basic_fewshot_qa_model, metric=answer_exact_match)

Average Metric: 2 / 15  (13.3): 100%|██████████| 15/15 [00:00<00:00, 105.67it/s]
/home/michael/.cache/pypoetry/virtualenvs/cs224u-projects-GPC47Gm_-py3.10/lib/python3.10/site-packages/dspy/evaluate/evaluate.py:142: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)


Average Metric: 2 / 15  (13.3%)


,question,example_answer,pred_answer,answer_exact_match
0,In 1517 who was Luther's bishop?,Albert of Mainz,Albert of Mainz,✔️ [True]
1,When was the construction that changed the Rhine's Delta?,20th Century,13th century,False
2,How many companies were registered in Warsaw in 2006?,"304,016","over 100,000",False
3,What is the CJEU's duty?,"to ""ensure that in the interpretation and application of the Treaties the law is observed""",interpret EU law,False
4,What would a teacher do for someone who is cocky?,deflate,challenge them,False


13.33

### Retrieval

The final major component of our systems is retrieval. When we defined `rm`, we connected to a remote ColBERT index and retriever system that we can now use for search.

The basic `dspy.retrieve` method returns only passages:

In [37]:
retriever = dspy.Retrieve(k=3)

In [38]:
passages = retriever("Which award did Gary Zukav's first book receive?")

In [39]:
passages

Prediction(
    passages=['Gary Zukav | Gary Zukav Gary Zukav (born October 17, 1942) is an American spiritual teacher and the author of four consecutive New York Times Best Sellers. Beginning in 1998, he appeared more than 30 times on "The Oprah Winfrey Show" to discuss transformation in human consciousness concepts presented in his book "The Seat of the Soul". His first book, "The Dancing Wu Li Masters" (1979), won a U.S. National Book Award. Gary Zukav was born in Port Arthur, Texas, and spent his early childhood in San Antonio and Houston. His family moved to Pittsburg, Kansas, while he was in fourth grade. In', 'The Dancing Wu Li Masters | The Dancing Wu Li Masters The Dancing Wu Li Masters is a 1979 book by Gary Zukav, a popular science work exploring modern physics, and quantum phenomena in particular. It was awarded a 1980 U.S. National Book Award in category of Science. Although it explores empirical topics in modern physics research, "The Dancing Wu Li Masters" gained attenti

If we need passages with scores and other metadata, we can call `rm` directly:

In [40]:
rm("Which award did Gary Zukav's first book receive?", k=1)

[{'pid': 7182463,
  'prob': 1.0,
  'rank': 1,
  'score': 24.77630615234375,
  'text': 'Gary Zukav | Gary Zukav Gary Zukav (born October 17, 1942) is an American spiritual teacher and the author of four consecutive New York Times Best Sellers. Beginning in 1998, he appeared more than 30 times on "The Oprah Winfrey Show" to discuss transformation in human consciousness concepts presented in his book "The Seat of the Soul". His first book, "The Dancing Wu Li Masters" (1979), won a U.S. National Book Award. Gary Zukav was born in Port Arthur, Texas, and spent his early childhood in San Antonio and Houston. His family moved to Pittsburg, Kansas, while he was in fourth grade. In',
  'long_text': 'Gary Zukav | Gary Zukav Gary Zukav (born October 17, 1942) is an American spiritual teacher and the author of four consecutive New York Times Best Sellers. Beginning in 1998, he appeared more than 30 times on "The Oprah Winfrey Show" to discuss transformation in human consciousness concepts presente

## Few-shot OpenQA with context

Let's build on the above core concepts to define a basic retrieval-augmented generation (RAG) program. This program solves the core task of few-shot OpenQA task and will serve as the basis for the homework questions:

We begin with a signature that takes context into account but is otherwise just like `BasicQASignature` above:

In [41]:
class ContextQASignature(dspy.Signature):
    __doc__ = """Answer questions with short factoid answers."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

And here is a complete program/system for the task:

In [42]:
class RAG(dspy.Module):
    def __init__(self, num_passages=1):
        super().__init__()
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.Predict(ContextQASignature)

    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

In [43]:
rag_model = RAG(num_passages=3)

In [44]:
rag_model(question="Which award did Gary Zukav's first book receive?")

Prediction(
    context=['Gary Zukav | Gary Zukav Gary Zukav (born October 17, 1942) is an American spiritual teacher and the author of four consecutive New York Times Best Sellers. Beginning in 1998, he appeared more than 30 times on "The Oprah Winfrey Show" to discuss transformation in human consciousness concepts presented in his book "The Seat of the Soul". His first book, "The Dancing Wu Li Masters" (1979), won a U.S. National Book Award. Gary Zukav was born in Port Arthur, Texas, and spent his early childhood in San Antonio and Houston. His family moved to Pittsburg, Kansas, while he was in fourth grade. In', 'The Dancing Wu Li Masters | The Dancing Wu Li Masters The Dancing Wu Li Masters is a 1979 book by Gary Zukav, a popular science work exploring modern physics, and quantum phenomena in particular. It was awarded a 1980 U.S. National Book Award in category of Science. Although it explores empirical topics in modern physics research, "The Dancing Wu Li Masters" gained attentio

An optional tiny evaluation:

In [45]:
tiny_evaluater(rag_model, metric=answer_exact_match)

Average Metric: 2 / 15  (13.3): 100%|██████████| 15/15 [00:00<00:00, 107.15it/s]

Average Metric: 2 / 15  (13.3%)



/home/michael/.cache/pypoetry/virtualenvs/cs224u-projects-GPC47Gm_-py3.10/lib/python3.10/site-packages/dspy/evaluate/evaluate.py:142: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)


,question,example_answer,context,pred_answer,answer_exact_match
0,In 1517 who was Luther's bishop?,Albert of Mainz,"[""Paul Speratus | Ellwangen, Priest of the Diocese of Augsburg). Early studies took him to Paris and Italy, as well as (probably) Freiburg and Vienna....",George of the Palatinate,False
1,When was the construction that changed the Rhine's Delta?,20th Century,"['Rhine | rivers and streams. Many rivers have been closed (""dammed"") and now serve as drainage channels for the numerous polders. The construction of Delta...",second half of the 20th Century,False
2,How many companies were registered in Warsaw in 2006?,"304,016","['Warsaw | such as Sydney, Istanbul, Amsterdam or Seoul. Warsaw, especially its city centre (""Śródmieście""), is home not only to many national institutions and government...","304,016",✔️ [True]
3,What is the CJEU's duty?,"to ""ensure that in the interpretation and application of the Treaties the law is observed""",['European Union law | elected by the judges for three years. While TEU article 19(3) says the Court of Justice is the ultimate court to...,To ensure the law is observed.,False
4,What would a teacher do for someone who is cocky?,deflate,"['Teacher | for the individual students accordingly. For example, an experienced teacher and parent described the place of a teacher in learning as follows: ""The...",Deflate the cocky.,False


13.33

## Question 1: Optimizing RAG [2 points]

We used `RAG` above as a zero-shot system. We could turn it into a few-shot system by using `LabeledFewShot` as we did in [the teleprompting section](#Teleprompting) above, but this may actually be problematic: if we randomly sample demonstrations with retrieved passages, we might be instructing the model with a lot of cases where the context passage isn't helping (and may actually be actively misleading the model). 

What we'd like to do is select demonstrations where the model gets the answer correct and the context passage does contain the answer. To do this, we will use the DSPy `BootstrapFewShot` optimizer. There are two steps for this: (1) defining a metric and (2) running the optimizer.

__Note__: The code for this question can be found in the DSPy tutorials, and you should feel free to make use of that code. The goal is to help you understand the design patterns and overall logic of optimizing DSPy programs.

__Task 1__: Complete `validate_context_and_answer` according to the specification in the docstring.

In [46]:
def validate_context_and_answer(example, pred, trace=None):
    """Return True if `example.answer` matches `pred.answer` according
    to `dspy.evaluate.answer_exact_match` and `pred.context` contains
    `example.answer` according to `dspy.evaluate.answer_exact_match`.

    Parameters
    ----------
    example: dspy.Example 
        with attributes `answer` and `context`
    pred: dspy.Example 
        with attributes `answer` and `context`
    trace : None (included for dspy internal compatibility)

    Returns
    -------
    bool

    """
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM and answer_PM




A test you can use to check your implementation:

In [47]:
def test_validate_context_and_answer(func):
    examples = [
        (
            dspy.Example(question="Q1", answer="B"),
            dspy.Prediction(question="Q1", context="A B C", answer="B"),
            True
        ),
        # Context doesn't contain answer, but predicted answer is correct.
        (
            dspy.Example(question="Q1", answer="D"),
            dspy.Prediction(question="Q1", context="A B C", answer="D"),
            False
        ),
        # Context contains answer, but predicted answer is not correct.
        (
            dspy.Example(question="Q1", answer="C"),
            dspy.Prediction(question="Q1", context="A B C", answer="D"),
            False
        )
    ]
    errcount = 0
    for ex, pred, result in examples:
        predicted = func(ex, pred, trace=None)
        if predicted != result:
            errcount += 1
            print(f"Error for `{func.__name__}`: "
                  f"Expected inputs\n\t{ex}\n\t{pred} to return {result}.")
    if errcount == 0:
        print(f"No errors detected for `{func.__name__}`")

In [48]:
test_validate_context_and_answer(validate_context_and_answer)

No errors detected for `validate_context_and_answer`


__Task 2__: Complete `bootstrap_optimize` according to the specification in the docstring.

In [49]:
from dspy.teleprompt import BootstrapFewShot

def bootstrap_optimize(model):
    """Use `BootstrapFewShot` to optimize `model`, with the metric set
    to `validate_context_and_answer` as defined above and default
    values for all other keyword arguments to `BootstrapFewShot`.

    Parameters
    ----------
    model: dspy.Module

    Returns
    -------
    dspy.Module, the optimized version of `model`

    """
    return BootstrapFewShot(metric=validate_context_and_answer).compile(model, trainset=dev_exs)
    




A test you can use to check your implementation:

In [50]:
def test_bootstrap_optimize(func):
    model = RAG()
    compiled = func(model)
    if not hasattr(compiled, "_compiled") or not compiled._compiled:
        print(f"Error for `{func.__name__}`: "
               "The return value is not a compiled program.")
        return None
    state = compiled.dump_state()
    if not state['generate_answer']['demos']:
        print(f"Error for `{func.__name__}`: "
               "The compiled program has no `demos`.")
        return None
    print(f"No errors detected for `{func.__name__}`")

In [51]:
test_bootstrap_optimize(bootstrap_optimize)

  4%|▍         | 9/200 [00:00<00:01, 119.10it/s]

Bootstrapped 4 full traces after 10 examples in round 0.
No errors detected for `bootstrap_optimize`


## Question 2: Multi-passage summarization [2 points]

The `dspy.Retrieve` layer in our `RAG` retrieves `k` passages, where `k` is under the control of the user. One hypothesis one might have is that it would be good to summarize these passages before using them as evidence. This seems especially likely to help in scenarios where the question can be answered only by synthesizing information across documents – it might be too much to ask the language model to do both synthesizing and answering in a single step.

The current question maps out a basic strategy for summarization. The heart of it is a new signature called `SummarizeSignature`. This can be used on its own with a simple `dspy.Predict` call, and we'll incorporate it into a RAG program in the next question.

For this question, though, your task is just to complete `SummarizeSignature`. The requirements are as follows:

1. A `__doc__` value that gives an instruction that seems to work well. You can decide what to say here.
2. A `dspy.InputField` named `context`. You can decide whether to use the `desc` parameter.
3. A `dspy.OutputField` named `summary`. You can decide whether to use the `desc` parameter.

In [52]:
class SummarizeSignature(dspy.Signature):
    __doc__ = """Reply with a short, clear, and concise summary of the key facts and statements from the following text."""

    context = dspy.InputField()
    summary = dspy.OutputField()




Here's a simple test that just checks for the required pieces in a basic way:

In [53]:
def test_SummarizeSignature(sigclass):
    fields = sigclass.fields
    expected_fieldnames = ['context', 'summary']
    fieldnames = sorted([field.input_variable for field in fields])
    errcount = 0
    if expected_fieldnames != fieldnames:
        errcount += 1
        print(f"Error for `{sigclass.__name__}`: "
              f"Expected fieldnames {expected_fieldnames}, got {fieldnames}.")
    if not sigclass.__doc__:
        errcount += 1
        print(f"Error for `{sigclass.__name__}`: No docstring specified.")
    if errcount == 0:
        print(f"No errors detected for `{sigclass.__name__}`")

In [54]:
test_SummarizeSignature(SummarizeSignature)

No errors detected for `SummarizeSignature`


Here is the simplest way to use `SummarizeSignature`:

In [55]:
summarizer = dspy.Predict(SummarizeSignature)

In [56]:
summarizer(context=retriever("Where is Guarani spoken?").passages)

Prediction(
    summary='Context: Information about the Guarani language and its dialects in South America.\nSummary: Guarani is an indigenous language spoken in Paraguay and neighboring countries, with different dialects like Eastern Bolivian Guarani and Mbyá in Argentina. The language is widely spoken by the Guarani people across South America, with over four million speakers in total.'
)

## Question 3: Summarizing RAG [2 points]

Your task for this question is to modify `RAG` as defined above so that the retrieved passages are summarized before being passed to `generate_answer`. 

Here is the `RAG` system copied from above with the class name changed to the one we will use for this new system. Your task is to add the summarization step. This should be very straightforward given the modular design that DSPy supports and encourages!

In [57]:
class SummarizingRAG(dspy.Module):
    def __init__(self, num_passages=3):
        # Please name your summarization later `summarize` so that we
        # can check for its presence.
        super().__init__()
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.summarize = dspy.Predict(SummarizeSignature)
        self.generate_answer = dspy.Predict(ContextQASignature)

    def forward(self, question):
        context = self.retrieve(question).passages
        summary = self.summarize(context=context).summary
        prediction = self.generate_answer(context=summary, question=question)
        return dspy.Prediction(context=summary, answer=prediction.answer)

A simple test for this design spec:

In [58]:
def test_SummarizingRAG(classname):
    model = classname(num_passages=3)
    errcount = 0
    if not hasattr(model, "summarize"):
        errcount += 1
        print(f"Error for `{classname.__name__}`: "
              f"Expected a layer called 'summarize'")
    context = model.retrieve("What are some foods?").passages
    pred = model("What are some foods?")
    if context == pred.context:
        errcount += 1
        print(f"Error for `{classname.__name__}`: "
              "The model seems to be using raw retrieved contexts "
              "for predictions rather than summarizing them.")
    if errcount == 0:
        print(f"No errors detected for `{classname.__name__}`")

In [59]:
test_SummarizingRAG(SummarizingRAG)

No errors detected for `SummarizingRAG`


Model usage:

In [60]:
summarizing_rag_model = SummarizingRAG()

In [61]:
summarizing_rag_model(question="Which award did Gary Zukav's first book receive?")

Prediction(
    context='Gary Zukav is an American spiritual teacher and author of four New York Times Best Sellers, known for discussing transformation in human consciousness on "The Oprah Winfrey Show". His first book, "The Dancing Wu Li Masters", won a U.S. National Book Award in 1979. Markus Zusak is an Australian author known for his books "The Book Thief" and "The Messenger", with "The Book Thief" being translated into over 30 languages and adapted into a film in 2013. Zusak has also discussed his unfinished novel "Bridge of Clay".',
    answer='U.S. National Book Award'
)

Note: if you decide to use `BootstrapFewShot` on this, be sure not to use the metric we defined above, which requires that the passage embeds the correct answer as a substring. Now that we are summarizing, this is unlikely to hold, even if the answers are good ones.

## Question 4: Your original system [3 points]

This question asks you to design your own few-shot OpenQA system. All of the code above can be used and modified for this, and the requirement is just that you try something new that goes beyond what we've done so far. 

Terms for the bake-off:

* You can make free use of SQuAD and other publicly available data.

* The LM must be an autoregressive language model. No trained QA components can be used. This includes general purpose LMs that have been fine-tuned for QA. (We have obviously waded into some vague territory here. The spirit of this is to make use of frozen, general-purpose models. We welcome questions about exactly how this is defined, since it could be instructive to explore this.)

Here are some ideas for the original system:

* We have relied almost entirely on `dspy.Predict`. Drop-in replacements include `dspy.ChainOfThought` and `dspy.ReAct`.

* We have used only one retriever. DSPy supports other retrieval mechanisms, including retrieval using [You.com](https://you.com/).

* DSPy includes additional optimizers. Two that are worth trying are `SignatureOptimizer` for automatic prompt exploration and `BootstrapFewShotWithRandomSearch`, which combines `LabeledFewShot` and `BootstrapFewShot`,

* Our one-step summarization procedure from Question 3 doesn't change the query to the retriever. We might want it to change as we gather evidence. This is a common design principle for multi-hop OpenQA systems.

__Original system instructions__:

In the cell below, please provide a brief technical description of your original system, so that the teaching team can gain an understanding of what it does. This will help us to understand your code and analyze all the submissions to identify patterns and strategies.

In [62]:
# PLEASE MAKE SURE TO INCLUDE THE FOLLOWING BETWEEN THE START AND STOP COMMENTS:
#   1) Textual description of your system.
#   2) The code for your original system.
# PLEASE MAKE SURE NOT TO DELETE OR EDIT THE START AND STOP COMMENTS

# START COMMENT: Enter your system description in this cell.

# 1) Textual description
# I implemented a more flexible version of the system defined so far with the following characteristics:
#     - I used gpt-3.5-turbo from OpenAI with default parameters where I implemented a retry mechanism since I experienced frequent read timeouts with the OpenAI API.
#     - Multi-hop with two hops as default and generation of retrieval query with extended context for each consecutive hop. 
#     - Ability to individually define each involved module (search query, summary, answer).
#     - Ability to select summarizing the whole context and/or summarizing each individual passage in the context.
#           Summarizing the whole context did not work well for me in my initial experiments so did not include it in the grid search.
#           I thought summarizing each individual passage in the context might help generating a good answer but that also proved inferior to delivering the whole context (see scores of grid search given below).
#     - I used BootstrapFewShotWithRandomSearch for optimization. 
#     - I stayed with the ColBERTv2 retriever since doing a basic grid search over the already configurable options took quite some time.
#     - I adapted the signature for summarization slightly based on some initial experiments. 
#     - Intially I wanted to do a full grid search over all available options and also including ReAct as a module. In my initial experiments, using ReAct had the downsides of a) taking longer and b) sometimes getting on a completely irrelevant track.
#       Therefore, I did not include ReAct in the grid search and narrowed down the options to 
        # a) Check, if summarizing individual passages in the context helps generating the answer.
        # b) Using Predict or ChainOfThought for generating the answer.
        # c) Always using Predict for summarization.
        # d) Alway using ChainOfThought for generating the queries for retrieving the context.
# Please find below the scores of my grid search using the dev evaluator already given in the notebook and 30 samples for optimization where the pattern for the identifier is as follows:
    # <summarize_retrieved_passages_per_hop>_<module_gen_answer>_<module_summarize>_<module_gen_query>. 
# The first element is a boolean indicating whether to summarize individual passages in the context and the remainder are the modules used for generating the answer, summarization, and generating the query for each hop.
# I used answer_passage_match as metric to not be too strict in the evaluation.

# True_<class 'dspy.predict.predict.Predict'>_<class 'dspy.predict.predict.Predict'>_<class 'dspy.predict.chain_of_thought.ChainOfThought'>: 50.5
# True_<class 'dspy.predict.chain_of_thought.ChainOfThought'>_<class 'dspy.predict.predict.Predict'>_<class 'dspy.predict.chain_of_thought.ChainOfThought'>: 49.5
# False_<class 'dspy.predict.predict.Predict'>_<class 'dspy.predict.predict.Predict'>_<class 'dspy.predict.chain_of_thought.ChainOfThought'>: 57.0
# False_<class 'dspy.predict.chain_of_thought.ChainOfThought'>_<class 'dspy.predict.predict.Predict'>_<class 'dspy.predict.chain_of_thought.ChainOfThought'>: 57.5

# Therefore, I used ChainOfThought for generating the queries as well as the answer and did not use summarization in the end. 
# I used this set of parameters to do another optimization with BootstrapFewShotWithRandomSearch using 200 samples insteach of 30 as in the grid search. 

# 2) Code
import itertools
import logging
import os
from random import sample 
import sys

import pickle
from dspy.evaluate import answer_passage_match
from dsp.utils import deduplicate
from tqdm import tqdm

from dspy.teleprompt import BootstrapFewShot, BootstrapFewShotWithRandomSearch

def flexible_optimize(model, config_optimizer=None, optimizer=BootstrapFewShotWithRandomSearch, metric=answer_passage_match, trainset=squad_train, devset=squad_dev, n_samples=30):
    if config_optimizer is None:
        config_optimizer = {}
    return optimizer(metric=metric, **config_optimizer).compile(model, trainset=sample(trainset, n_samples))

log_format = '%(funcName)s - %(levelname)s - %(message)s'
logging.basicConfig(format=log_format, level=logging.INFO, stream=sys.stdout, force=True)
logger = logging.getLogger(__name__)

class GenerateSearchQuery(dspy.Signature):
    """Write a short and concise search query that will help answer a complex question."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    query = dspy.OutputField()
    
class SummarizeSignature(dspy.Signature):
    # __doc__ = """Reply with a concise summary of the key facts and statements from the following text."""
    __doc__ = """Summarize the following text in one paragraph focusing on key facts and statements."""

    context = dspy.InputField()
    summary = dspy.OutputField()
    
class ContextQASignature(dspy.Signature):
    __doc__ = """Answer questions with short factoid answers."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")
    
class FlexibleRAG(dspy.Module):
    def __init__(self, 
                 passages_per_hop=3, 
                 max_hops=2,
                 summarize_retrieved_passages_per_hop=True,
                 summarize_whole_context=False,
                 module_gen_answer=dspy.Predict, 
                 signature_gen_answer=ContextQASignature,
                 module_summarize=dspy.Predict,
                 signature_summarize=SummarizeSignature,
                 module_gen_query=dspy.Predict,
                 signature_gen_query=GenerateSearchQuery,
                 loglevel=logging.INFO):
        super().__init__()    
        self.logger = logging.getLogger(__name__)
        self.logger.setLevel(loglevel)    
        self.max_hops = max_hops
        self.summarize_retrieved_passages_per_hop = summarize_retrieved_passages_per_hop
        self.summarize_whole_context = summarize_whole_context
        self.summarize = None
        self.generate_query = None
        if module_summarize is not None and signature_summarize is not None:
            self.summarize = module_summarize(signature_summarize)
        if module_gen_query is not None and signature_gen_query is not None:
            self.generate_query = [module_gen_query(signature_gen_query) for _ in range(max_hops)]            
        self.retrieve = dspy.Retrieve(k=passages_per_hop)
        self.generate_answer = module_gen_answer(signature_gen_answer)
    
    def get_context(self, question):
        if self.generate_query is not None:
            context = []
            for hop in range(self.max_hops):
                self.logger.debug(f'{hop=}')
                query = self.generate_query[hop](context=context, question=question).query            
                self.logger.debug(f'{query=}')
                passages = self.retrieve(query).passages
                if self.summarize_retrieved_passages_per_hop and self.summarize is not None:
                    # Summarize every passage to ease generation of answer and reduce context size
                    passages = [self.get_summary(passage) for passage in passages]                   
                context = deduplicate(context + passages)
                self.logger.debug(f'{context=}')
        else:
            context = self.retrieve(question).passages
        return context
            
    def get_summary(self, context):
        if self.summarize is not None:
            summary = self.summarize(context=context).summary
            self.logger.debug(f'{context=}')
            self.logger.debug(f'{summary=}')
            return summary
        else:
            return context        

    def forward(self, question):
        context = self.get_context(question=question)  
        if self.summarize_whole_context and self.summarize is not None:  
            summary = self.get_summary(context=context)
        else:
            summary = context
        prediction = self.generate_answer(context=summary, question=question)        
        self.logger.debug(f'{question=}')
        self.logger.debug(f'{context=}')
        self.logger.debug(f'{summary=}')
        self.logger.debug(f'{prediction.answer=}')
        return dspy.Prediction(context=summary, answer=prediction.answer)    
    
model_selection = False
config_optimizer = dict(max_bootstrapped_demos=5, max_labeled_demos=5, max_rounds=1, num_candidate_programs=3)

if model_selection:
    n_retries = 50    
    # options_module_gen_answer=[dspy.Predict, dspy.ReAct, dspy.ChainOfThought]
    # options_module_summarize=[dspy.Predict, dspy.ReAct, dspy.ChainOfThought]
    # options_module_gen_query=[dspy.Predict, dspy.ReAct, dspy.ChainOfThought]
    options_summarize_retrieved_passages_per_hop=[True, False]
    options_module_gen_answer=[dspy.Predict, dspy.ChainOfThought]
    options_module_summarize=[dspy.Predict]
    options_module_gen_query=[dspy.ChainOfThought]
    grid = itertools.product(options_summarize_retrieved_passages_per_hop, options_module_gen_answer, options_module_summarize, options_module_gen_query)
    
    if os.path.exists('models.pkl'):
        with open('models.pkl', 'rb') as file:
            models = pickle.load(file)
    else:
        models = {}
    if os.path.exists('results.pkl'):
        with open('results.pkl', 'rb') as file:
            results = pickle.load(file)
    else:
        results = {}
    def grid_search(grid, models, results):
        try:    
            for summarize_retrieved_passages_per_hop, module_gen_answer, module_summarize, module_gen_query in tqdm(grid):
                logger.info(f'Optimizing model with settings\n{summarize_retrieved_passages_per_hop=}, {module_gen_answer=}, {module_summarize=}, {module_gen_query=}')
                id = f'{str(summarize_retrieved_passages_per_hop)}_{str(module_gen_answer)}_{str(module_summarize)}_{str(module_gen_query)}'
                if id not in models:
                    models[id] = flexible_optimize(model=FlexibleRAG(summarize_retrieved_passages_per_hop=summarize_retrieved_passages_per_hop, 
                                                                    module_gen_answer=module_gen_answer,
                                                                    module_gen_query=module_gen_query,
                                                                    module_summarize=module_summarize,
                                                                    passages_per_hop=3),
                                                optimizer=BootstrapFewShotWithRandomSearch,
                                                config_optimizer=config_optimizer,
                                                n_samples=30,
                                                metric=answer_passage_match,)
                    
                    with open('models.pkl', 'wb') as file:
                        pickle.dump(models, file)
                logger.info('Evaluating model')
                if id not in results:
                    results[id] = dev_evaluater(models[id], metric=answer_passage_match, display=True, display_table=10, return_outputs=True)
                    logger.info(f'Current results\n{results}')
                    with open('results.pkl', 'wb') as file:
                        pickle.dump(results, file)
        except Exception as e:
            logger.error(e)
            n_retries = n_retries - 1
            logger.info(f'Retrying... {n_retries} remaining')
            if n_retries >=0:
                grid_search(grid, models, results)
            else:
                raise e
    grid_search(grid, models, results)
        
best_set_of_parameters = {
    'summarize_retrieved_passages_per_hop': False, 
    'module_gen_answer': dspy.ChainOfThought, 
    'module_summarize': dspy.Predict, 
    'module_gen_query': dspy.ChainOfThought
}
resulting_model = flexible_optimize(model=FlexibleRAG(summarize_retrieved_passages_per_hop=best_set_of_parameters['summarize_retrieved_passages_per_hop'], 
                                                                    module_gen_answer=best_set_of_parameters['module_gen_answer'],
                                                                    module_gen_query=best_set_of_parameters['module_gen_query'],
                                                                    module_summarize=best_set_of_parameters['module_summarize'],
                                                                    passages_per_hop=3),
                                                optimizer=BootstrapFewShotWithRandomSearch,
                                                config_optimizer=config_optimizer,
                                                n_samples=200,
                                                metric=answer_passage_match,)
with open('resulting_model.pkl', 'wb') as file:
    pickle.dump(resulting_model, file)
        
    
# STOP COMMENT: Please do not remove this comment.

Going to sample between 1 and 5 traces per predictor.
Will attempt to train 3 candidate sets.


Average Metric: 120 / 200  (60.0): 100%|██████████| 200/200 [00:07<00:00, 28.37it/s]


Average Metric: 120 / 200  (60.0%)
Score: 60.0 for set: [0, 0, 0, 0]
New best score: 60.0 for seed -3
Scores so far: [60.0]
Best score: 60.0


Average Metric: 120 / 200  (60.0): 100%|██████████| 200/200 [00:07<00:00, 28.09it/s]


Average Metric: 120 / 200  (60.0%)
Score: 60.0 for set: [5, 5, 5, 5]
Scores so far: [60.0, 60.0]
Best score: 60.0


...

log_info - INFO - error_code=rate_limit_exceeded error_message='Rate limit reached for gpt-3.5-turbo in organization org-VjuFA3ZuxujWIoGp9RVspxfT on tokens per min (TPM): Limit 60000, Used 58190, Requested 1914. Please try again in 104ms. Visit https://platform.openai.com/account/rate-limits to learn more.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
_log_backoff - INFO - Backing off request(...) for 0.9s (openai.error.RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-VjuFA3ZuxujWIoGp9RVspxfT on tokens per min (TPM): Limit 60000, Used 58190, Requested 1914. Please try again in 104ms. Visit https://platform.openai.com/account/rate-limits to learn more.)
Backing off 0.9 seconds after 1 tries calling function <function GPT3.request at 0x7fc1ed3c7f40> with kwargs {}


Average Metric: 130 / 194  (67.0):  97%|█████████▋| 194/200 [09:07<00:17,  2.84s/it]

log_info - INFO - error_code=rate_limit_exceeded error_message='Rate limit reached for gpt-3.5-turbo in organization org-VjuFA3ZuxujWIoGp9RVspxfT on tokens per min (TPM): Limit 60000, Used 58594, Requested 1937. Please try again in 531ms. Visit https://platform.openai.com/account/rate-limits to learn more.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
_log_backoff - INFO - Backing off request(...) for 0.9s (openai.error.RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-VjuFA3ZuxujWIoGp9RVspxfT on tokens per min (TPM): Limit 60000, Used 58594, Requested 1937. Please try again in 531ms. Visit https://platform.openai.com/account/rate-limits to learn more.)
Backing off 0.9 seconds after 1 tries calling function <function GPT3.request at 0x7fc1ed3c7f40> with kwargs {}


Average Metric: 134 / 200  (67.0): 100%|██████████| 200/200 [14:58<00:00,  4.49s/it]

Average Metric: 134 / 200  (67.0%)
Score: 67.0 for set: [5, 5, 5, 5]
New best score: 67.0 for seed 2
Scores so far: [60.0, 60.0, 59.0, 62.0, 61.0, 67.0]
Best score: 67.0
Average of max per entry across top 1 scores: 0.67
Average of max per entry across top 2 scores: 0.845
Average of max per entry across top 3 scores: 0.895
Average of max per entry across top 5 scores: 0.97
Average of max per entry across top 8 scores: 0.98
Average of max per entry across top 9999 scores: 0.98
6 candidate programs found.


## Question 5: Bakeoff entry [1 point]

For the bake-off, you simply need to be able to run your system on the file 

```data/openqa/cs224u-openqa-test-unlabeled.txt```

The following code should download it for you if necessary:

In [67]:
import wget

if not os.path.exists(os.path.join("data", "openqa", "cs224u-openqa-test-unlabeled.txt")):
    os.makedirs(os.path.join('data', 'openqa'), exist_ok=True)
    wget.download('https://web.stanford.edu/class/cs224u/data/cs224u-openqa-test-unlabeled.txt', out='data/openqa/')

If the above fails, you can just download https://web.stanford.edu/class/cs224u/data/cs224u-openqa-test-unlabeled.txt and place it in `data/openqa`.

This file contains only questions. The starter code below will help you structure this. It writes a file "cs224u-openqa-bakeoff-entry.json" to the current directory. That file should be uploaded as-is. Please do not change its name.

In [68]:
import json
import tqdm

def create_bakeoff_submission(model):
    """"
    The argument `model` is a `dspy.Module`. The return value of its
    `forward` method must have an `answer` attribute.
    """

    filename = os.path.join("data", "openqa", "cs224u-openqa-test-unlabeled.txt")

    # This should become a mapping from questions (str) to response
    # dicts from your system.
    gens = {}

    with open(filename) as f:
        questions = f.read().splitlines()

    # Here we loop over the questions, run the system `model`, and
    # store its `answer` value as the prediction:
    for question in tqdm.tqdm(questions):
        gens[question] = model(question=question).answer

    # Quick tests we advise you to run:
    # 1. Make sure `gens` is a dict with the questions as the keys:
    assert all(question in gens for q in questions)
    # 2. Make sure the values are str:
    assert all(isinstance(d, str) for d in gens.values())

    # And finally the output file:
    with open("cs224u-openqa-bakeoff-entry.json", "wt") as f:
        json.dump(gens, f, indent=4)

Here's what it looks like to evaluate our first program, `basic_qa_model`, on the bakeoff data:

In [69]:
# create_bakeoff_submission(basic_qa_model)

In [70]:
create_bakeoff_submission(resulting_model)

100%|██████████| 400/400 [26:27<00:00,  3.97s/it]
